# Exemplo 03 - Usando Tools com LangGraph

## 1 - Carrega Bibliotecas

In [1]:
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool
from dotenv import load_dotenv

## 2 - Carrega API Key

In [2]:
from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv())
llm_model = ChatGroq(model_name='llama-3.3-70b-versatile',temperature=0)

## 3 - Define o prompt do sistema

In [3]:
system_message = SystemMessage(content = """
Você é um assistente. Se o usuário pedir contas, use
a ferramenta 'somar'. Caso contrário, apenas responda normalmente
"""
)

## 4 - Definindo a ferramenta de soma

In [4]:
@tool("somar")
def somar(valores: str) -> str:
    """Soma dois números separados por vírgula"""
    try:
        a, b = map(float, valores.split(","))
        return str(a + b)
    except Exception as e:
        return f"Erro ao somar: {str(e)}"

## 5 - Criação do Agente com LangGraph

In [5]:
tools = [somar]
graph = create_react_agent(
    model=llm_model,
    tools=tools,
    prompt=system_message
)
export_graph = graph

## 6 - Extrair a resposta final

In [6]:
def extrair_resposta_final(result):
    ai_messages = [m for m in result["messages"] if isinstance(m, AIMessage) and m.content]
    if ai_messages:
        return ai_messages[-1].content
    else:
        return "Nenhuma resposta final encontrada"

## 7 - Testando o Agente

In [7]:
entrada1 = HumanMessage(content="Quanto é 8 + 5?")
result1 = export_graph.invoke({"messages":[entrada1]})
for m in result1["messages"]:
    print(m)
resposta_texto_1 = extrair_resposta_final(result1)
print("Resposta 1: ", resposta_texto_1)
print()

content='Quanto é 8 + 5?' additional_kwargs={} response_metadata={} id='b458ce01-5f4b-4021-adfa-92dfe934224a'
content='' additional_kwargs={'tool_calls': [{'id': 'nd3j1dmtz', 'function': {'arguments': '{"valores":"8,5"}', 'name': 'somar'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 266, 'total_tokens': 283, 'completion_time': 0.033429563, 'prompt_time': 0.039816693, 'queue_time': 0.268507156, 'total_time': 0.073246256}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_2ddfbb0da0', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--5638ff86-9495-4c09-9925-0b382fe3a2e1-0' tool_calls=[{'name': 'somar', 'args': {'valores': '8,5'}, 'id': 'nd3j1dmtz', 'type': 'tool_call'}] usage_metadata={'input_tokens': 266, 'output_tokens': 17, 'total_tokens': 283}
content='13.0' name='somar' id='b7faf096-2e9f-46d6-94c9-830b67c09bab' tool_call_id='nd3j1dmtz'
content='O resultado da soma de 8 e 5 é 

In [8]:
entrada2 = HumanMessage(content="Quem pintou a Monalisa?")
result2 = export_graph.invoke({"messages":[entrada2]})
for m in result2["messages"]:
    print(m)
resposta_texto_2 = extrair_resposta_final(result2)
print("Resposta 2: ", resposta_texto_2)

content='Quem pintou a Monalisa?' additional_kwargs={} response_metadata={} id='ae7e34c6-6e17-4a22-aaca-e2c74e0a2ead'
content='A Monalisa foi pintada por Leonardo da Vinci, um renomado artista e polímata italiano, no início do século XVI. A pintura, criada entre 1503 e 1506, é uma das obras mais famosas do mundo e está exposta no Museu do Louvre, em Paris.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 71, 'prompt_tokens': 266, 'total_tokens': 337, 'completion_time': 0.214411588, 'prompt_time': 0.026435028, 'queue_time': 0.269855202, 'total_time': 0.240846616}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_2ddfbb0da0', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None} id='run--ef43ee75-28af-400f-9fa6-2ff22421024f-0' usage_metadata={'input_tokens': 266, 'output_tokens': 71, 'total_tokens': 337}
Resposta 2:  A Monalisa foi pintada por Leonardo da Vinci, um renomado artista e polímata italiano, no início do século X